In [1]:
import pandas as pd
import re
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words= set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger')
data=pd.read_json("/content/drive/My Drive/NLP/exam/JEOPARDY_QUESTIONS1.json",encoding='utf-8') 
data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680
...,...,...,...,...,...,...,...
216925,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,$2000,Turandot,Double Jeopardy!,4999
216926,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,$2000,a titmouse,Double Jeopardy!,4999
216927,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",$2000,Clive Barker,Double Jeopardy!,4999
216928,QUOTATIONS,2006-05-11,"'From Ft. Sill, Okla. he made the plea, Arizon...",$2000,Geronimo,Double Jeopardy!,4999


In [2]:
#Q1
import json
config = json.loads(open('/content/drive/My Drive/NLP/exam/JEOPARDY_QUESTIONS1.json').read())

from collections import Counter  
ct=Counter([x['category'] for x in config if x.get('category')])#\'category\':.*?(?=,)
ct=ct.most_common(len(ct))
ct[0:10]

[('BEFORE & AFTER', 547),
 ('SCIENCE', 519),
 ('LITERATURE', 496),
 ('AMERICAN HISTORY', 418),
 ('POTPOURRI', 401),
 ('WORLD HISTORY', 377),
 ('WORD ORIGINS', 371),
 ('COLLEGES & UNIVERSITIES', 351),
 ('HISTORY', 349),
 ('SPORTS', 342)]

In [3]:
#Q2
data1=data
data1['value1'] = data['value'].str.extract('\$(\d+)', expand=False).astype(float)#clean digital
data1["length"]= data["question"].str.len() 
data1=data1.fillna(0)
from scipy.stats import pearsonr
x = np.asarray(data1['value1'])
y = np.asarray(data1['length'])
print(pearsonr(x,y))#r=0.1112,p-value=0, a liitle connection?

(0.10903455652403451, 0.0)


In [4]:
data

,category,air_date,question,value,answer,round,show_number,value1,length
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680,200.0,98
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680,200.0,109
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680,200.0,90
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680,200.0,86
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680,200.0,106
...,...,...,...,...,...,...,...,...,...
216925,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,$2000,Turandot,Double Jeopardy!,4999,2000.0,76
216926,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,$2000,a titmouse,Double Jeopardy!,4999,2000.0,105
216927,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",$2000,Clive Barker,Double Jeopardy!,4999,2000.0,107
216928,QUOTATIONS,2006-05-11,"'From Ft. Sill, Okla. he made the plea, Arizon...",$2000,Geronimo,Double Jeopardy!,4999,2000.0,120


In [5]:
def getlen(x):
  x=x.lower()
  tags = set(['JJ', 'JJR', 'JJS'])
  x=word_tokenize(x)
  #print(x)
  pos_tags =nltk.pos_tag(x)
  #print(pos_tags)
  ret = []
  for word,pos in pos_tags:
    if (pos in tags and re.match("^[a|e|i|o|u]",word)):
      ret.append(word)
  #print(ret)
  return(len(ret))
data2006=data.loc[(data['air_date'].str.startswith('2006'))].reset_index(drop=True)
data2006["adj_vow"]=data2006["question"].apply(lambda x :getlen(x))
data2006


,category,air_date,question,value,answer,round,show_number,value1,length,adj_vow
0,THE CIVIL WAR,2006-02-06,'His first act after being sworn in as preside...,$200,Jefferson Davis,Jeopardy!,4931,200.0,119,0
1,CELEBS,2006-02-06,"'On Sept. 14, 2005 she gave birth to Sean Pres...",$200,Britney Spears,Jeopardy!,4931,200.0,60,0
2,WHAT'S IN A NAME?,2006-02-06,"'Yeah, baby! Meaning ""magnificent"", this Texa...",$200,Austin,Jeopardy!,4931,200.0,90,0
3,EMOTICONS,2006-02-06,';-)<br />Ocular act that sends a signal',$200,winking,Jeopardy!,4931,200.0,41,1
4,FLAG 'EM DOWN,2006-02-06,'The Alamo is located in this city & is depict...,$200,San Antonio,Jeopardy!,4931,200.0,61,0
...,...,...,...,...,...,...,...,...,...,...
13721,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,$2000,Turandot,Double Jeopardy!,4999,2000.0,76,0
13722,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,$2000,a titmouse,Double Jeopardy!,4999,2000.0,105,0
13723,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",$2000,Clive Barker,Double Jeopardy!,4999,2000.0,107,1
13724,QUOTATIONS,2006-05-11,"'From Ft. Sill, Okla. he made the plea, Arizon...",$2000,Geronimo,Double Jeopardy!,4999,2000.0,120,0


In [6]:
total=sum(data2006["adj_vow"])
total

3860

In [0]:
#Q4
knntest=data[['question','category']]

#may not use
def clean_text(croup):
  x=sent_tokenize(croup)
  clean_senteces=pd.Series(x).str.replace("[^a-zA-Z]"," ")
  clean_senteces=[s.lower() for s in clean_senteces]
  text_no_sp=[]
  for r in clean_senteces:
    tmp=r.split()
    sen_new=" ".join([i for i in tmp if i not in stop_words ])
    if len(sen_new)>0:
      text_no_sp.append(sen_new)
  text_no_sp="".join(text_no_sp)
  return(text_no_sp)


In [23]:
#knntest["question1"]=knntest["question"].apply(lambda x :clean_text(x))
#knntest=knntest.loc[knntest['category']=="SCIENCE"].reset_index(drop=True)
#df.loc[df['columnName']=='the value']
#knntest_label=np.asarray(knntest["category"])
knn=knntest.loc[knntest['category'].isin(['SCIENCE','LITERATURE'])  ].reset_index(drop=True)

knntest_label=np.asarray(knn["category"])
len(knn)

1015

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(stop_words=set(stop_words))
knntest_set = v.fit_transform(knn["question"])


In [0]:
from sklearn import neighbors     
knn = neighbors.KNeighborsClassifier(n_neighbors=3, algorithm='auto', weights='distance', n_jobs=1)
knn.fit(knntest_set[0:800],knntest_label[0:800])
pred_label=knn.predict(knntest_set[800:900])
insamp=knn.predict(knntest_set[0:800])


In [21]:
from sklearn import metrics
def metrics_result(actual, predict):
  print('precision:{0:.3f}'.format(metrics.precision_score(actual, predict, average='weighted')))
  print('recall:{0:0.3f}'.format(metrics.recall_score(actual, predict, average='weighted')))
  print('f1-score:{0:.3f}'.format(metrics.f1_score(actual, predict, average='weighted')))
print("train:0~800,test:800~900")
print("=======out of sample========")
metrics_result(knntest_label[800:900], pred_label)
print("==========in sample==========")
metrics_result(knntest_label[0:800], insamp)

train:0~800,test:800~900
=======out of sample========
precision:0.912
recall:0.910
f1-score:0.910
==========in sample==========
precision:1.000
recall:1.000
f1-score:1.000


In [0]:
rilabel=[]
for i in knntest_label:
  if i=="SCIENCE":
    rilabel.append(0)
  else:
    rilabel.append(1)
from sklearn.linear_model import RidgeClassifier
clf =RidgeClassifier()
clf.fit(knntest_set[0:800],rilabel[0:800]) 
clfpre=clf.predict(knntest_set[800:900])
incl=clf.predict(knntest_set[0:800])

In [19]:
print("train:0~800,test:800~900")
print("=======out of sample========")
metrics_result(rilabel[800:900], clfpre)
print("==========in sample==========")
metrics_result(rilabel[0:800], incl)


train:0~800,test:800~900
=======out of sample========
precision:0.938
recall:0.930
f1-score:0.929
==========in sample==========
precision:1.000
recall:1.000
f1-score:1.000
